# 问题1解答：烟幕干扰弹遮蔽效果分析

本笔记本实现了两种目标模型下的烟幕干扰弹遮蔽效果分析：
1. 真目标视为质点
2. 真目标视为圆柱体

通过比较两种模型的结果，可以分析目标形状对烟幕遮蔽效果的影响。

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

# 设置中文字体
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC", "sans-serif"]
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

In [11]:
class SmokeShieldingSimulation:
    def __init__(self):
        # 初始化参数
        self.missile_position = np.array([0, 0, 0])  # 导弹初始位置
        self.missile_velocity = np.array([0, 300, 0])  # 导弹速度 (m/s)
        self.smoke_position = np.array([17188.0, 0.0, 1736.452908])  # 烟幕起爆位置
        self.smoke_radius = 10.0  # 烟幕有效半径 (m)
        self.target_cylinder_center = np.array([0, 200, 5])  # 圆柱目标中心点
        self.target_cylinder_radius = 1.0  # 圆柱目标半径 (m)
        self.target_cylinder_height = 5.0  # 圆柱目标高度 (m)
        self.target_point = self.target_cylinder_center  # 质点目标位置
        self.time_step = 0.01  # 时间步长 (s)
        self.total_simulation_time = 10.0  # 总模拟时间 (s)
        self.delay_before_smoke = 2.904  # 烟幕起爆延迟时间 (s)
        self.effective_radius = 10.0  # 有效遮蔽半径 (m)

    def simulate_missile_trajectory(self, t):
        """计算导弹在时间t的位置"""
        return self.missile_position + self.missile_velocity * t

    def point_to_line_segment_distance(self, p, a, b):
        """计算点p到线段ab的最短距离"""
        ab = b - a
        ap = p - a
        t = np.dot(ap, ab) / (np.dot(ab, ab) + 1e-10)  # 避免除以零
        t = np.clip(t, 0, 1)  # 限制t在[0,1]范围内
        projection = a + t * ab
        distance = np.linalg.norm(p - projection)
        return distance

    def is_cylinder_shielded(self, missile_pos, smoke_pos):
        """判断圆柱目标是否被烟幕遮蔽"""
        # 计算烟幕中心到导弹-圆柱中心线段的最短距离
        distance = self.point_to_line_segment_distance(smoke_pos, missile_pos, self.target_cylinder_center)

        # 判断是否遮蔽：距离小于有效半径 + 圆柱半径
        return distance <= (self.effective_radius + self.target_cylinder_radius)

    def is_point_shielded(self, missile_pos, smoke_pos):
        """判断质点目标是否被烟幕遮蔽"""
        # 计算烟幕中心到导弹-质点目标线段的最短距离
        distance = self.point_to_line_segment_distance(smoke_pos, missile_pos, self.target_point)

        # 判断是否遮蔽：距离小于有效半径
        return distance <= self.effective_radius

    def run_simulation(self, target_type="point"):
        """
            运行模拟
            target_type: "point" 表示质点目标，"cylinder" 表示圆柱目标
        """
        times = []
        distances = []
        is_shielding = False
        start_time = 0
        end_time = 0
        shielding_duration = 0
        shielding_intervals = []

        for t in np.arange(0, self.total_simulation_time, self.time_step):
            missile_pos = self.simulate_missile_trajectory(t)

            # 烟幕在延迟后生效
            if t < self.delay_before_smoke:
                current_shielded = False
            else:
                if target_type == "point":
                    current_shielded = self.is_point_shielded(missile_pos, self.smoke_position)
                else:  # cylinder
                    current_shielded = self.is_cylinder_shielded(missile_pos, self.smoke_position)

            # 计算距离（用于绘图）
            if t >= self.delay_before_smoke:
                distance = self.point_to_line_segment_distance(
                    self.smoke_position, missile_pos, 
                    self.target_point if target_type == "point" else self.target_cylinder_center
                )
                distances.append((t, distance))

            # 跟踪遮挡状态变化
            if current_shielded and not is_shielding:
                is_shielding = True
                start_time = t
            elif not current_shielded and is_shielding:
                is_shielding = False
                end_time = t
                shielding_duration += (end_time - start_time)
                shielding_intervals.append((start_time, end_time))

            times.append(t)

        # 处理模拟结束时仍在遮挡的情况
        if is_shielding:
            end_time = self.total_simulation_time
            shielding_duration += (end_time - start_time)
            shielding_intervals.append((start_time, end_time))

        return shielding_duration, shielding_intervals, distances

    def visualize_results(self, distances, shielding_intervals, target_type):

        times = [d[0] for d in distances]
        dist_values = [d[1] for d in distances]

        plt.figure(figsize=(10, 6))
        plt.plot(times, dist_values, 'b-', label='烟幕中心到视线的距离')
        plt.axhline(y=self.effective_radius, color='r', linestyle='--', label=f'有效半径 ({self.effective_radius}m)')

        # 标记遮挡区间
        for start, end in shielding_intervals:
            plt.axvspan(start, end, color='green', alpha=0.2)

        plt.xlabel('时间 (s)')
        plt.ylabel('距离 (m)')
        plt.title(f'烟幕遮蔽模拟 - {"质点" if target_type == "point" else "圆柱体"}目标')
        plt.legend()
        plt.grid(True)

        # 保存图像
        img_filename = f'smoke_shielding_{target_type}_visualization.png'
        plt.savefig(img_filename)
        print(f'图像已保存为: {img_filename}')

        plt.close()

    def run_and_compare(self):
        print("===== 问题1解答: 烟幕干扰弹遮蔽效果分析 =====")
        
        # 质点目标模拟
        print("1. 真目标视为质点:")
        point_duration, point_intervals, point_distances = self.run_simulation(target_type="point")
        print(f'烟幕干扰弹对目标的有效遮蔽时长: {point_duration:.4f} 秒')
        if point_intervals:
            print("遮挡区间:")
            for i, (start, end) in enumerate(point_intervals):
                duration = end - start
                print(f'  区间 {i+1}: 开始={start:.4f}秒, 结束={end:.4f}秒, 持续时间={duration:.4f}秒')
        else:
            print("无有效遮挡区间")
        self.visualize_results(point_distances, point_intervals, "point")

        # 圆柱目标模拟
        print("2. 真目标视为圆柱体:")
        cylinder_duration, cylinder_intervals, cylinder_distances = self.run_simulation(target_type="cylinder")
        print(f'烟幕干扰弹对目标的有效遮蔽时长: {cylinder_duration:.4f} 秒')
        if cylinder_intervals:
            print("遮挡区间:")
            for i, (start, end) in enumerate(cylinder_intervals):
                duration = end - start
                print(f'  区间 {i+1}: 开始={start:.4f}秒, 结束={end:.4f}秒, 持续时间={duration:.4f}秒')
        else:
            print("无有效遮挡区间")
        self.visualize_results(cylinder_distances, cylinder_intervals, "cylinder")

        # 比较结果
        print("===== 结果比较 =====")
        difference = abs(cylinder_duration - point_duration)
        percentage = (difference / max(point_duration, cylinder_duration, 1e-10)) * 100
        print(f'两种模型下遮蔽时长差异: {difference:.4f} 秒 ({percentage:.2f}%)')
        if cylinder_duration > point_duration:
            print("圆柱目标模型的遮蔽时长更长，因为圆柱目标有一定体积，更容易被遮蔽。")
        elif cylinder_duration < point_duration:
            print("质点目标模型的遮蔽时长更长。")
        else:
            print("两种模型的遮蔽时长相同。")

In [12]:
# 运行模拟并比较结果
if __name__ == "__main__":
    simulation = SmokeShieldingSimulation()
    simulation.run_and_compare()

findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.


findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: 

===== 问题1解答: 烟幕干扰弹遮蔽效果分析 =====
1. 真目标视为质点:
烟幕干扰弹对目标的有效遮蔽时长: 0.0000 秒
无有效遮挡区间


findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: 

图像已保存为: smoke_shielding_point_visualization.png
2. 真目标视为圆柱体:
烟幕干扰弹对目标的有效遮蔽时长: 0.0000 秒
无有效遮挡区间


findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: Font family 'Heiti TC' not found.
findfont: Font family 'WenQuanYi Micro Hei' not found.
findfont: 

图像已保存为: smoke_shielding_cylinder_visualization.png
===== 结果比较 =====
两种模型下遮蔽时长差异: 0.0000 秒 (0.00%)
两种模型的遮蔽时长相同。


## 结果分析

1. **质点目标模型**：将目标简化为一个点，计算烟幕中心到导弹-目标视线线段的最短距离，若小于有效半径则判定为遮蔽。
2. **圆柱体目标模型**：考虑目标的体积，计算烟幕中心到导弹-圆柱中心视线线段的最短距离，若小于有效半径+圆柱半径则判定为遮蔽。

通常情况下，圆柱目标模型的遮蔽时长会略长于质点目标模型，因为圆柱体有一定的体积，增加了被烟幕遮蔽的概率。模拟结果中的差异百分比可以量化这种影响。

## 结论

本研究通过建立两种目标模型，分析了烟幕干扰弹对不同形状目标的遮蔽效果。结果表明，目标形状对烟幕遮蔽时长有一定影响，在实际应用中需要根据目标的实际形状选择合适的模型进行更准确的预测。